In [ ]:
from datasets import load_dataset, load_metric
import datasets
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import random
import torch
from torch.optim import Adam

import transformers
from transformers import AdamW,AutoTokenizer, AutoModelForSequenceClassification,get_linear_schedule_with_warmup, TrainingArguments, Trainer
from torch.optim import Adadelta, Adam, SGD, RMSprop, AdamW
import wandb
#from transformers.trainer_utils import A
#%pip install scikit-learn

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
#Training Arguments
model_checkpoint = "microsoft/mpnet-base"#"symanto/mpnet-base-snli-mnli" got 87% on MNLI
MODEL_NAME = model_checkpoint.split("/")[-1]
MODEL = AutoModelForSequenceClassification.from_pretrained
METRIC_NAME = "accuracy"
BATCHSIZE = 4
OPTIMIZER=AdamW #AdamW, SGD, Adadelta
EPOCHNUM = 0.20
WEIGHT_DECAY = 0.8
TOKENIZER=AutoTokenizer.from_pretrained
dataset = load_dataset("glue", "mnli")
metric = load_metric('glue', "mnli")
num_labels = 3 # for mnli
optimizer_name = "AdamW"

In [ ]:
tokenizer = TOKENIZER(model_checkpoint, use_fast=True) # for mnli
tokenizer("Hello, this one sentence!", "And this sentence goes with it.", truncation=True) # for mnli
MODEL=MODEL(model_checkpoint, num_labels=num_labels) # for mnli

{'input_ids': [0, 7596, 1014, 2027, 2032, 6255, 1003, 2, 2, 2002, 2027, 6255, 3636, 2011, 2013, 1016, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
sentence1_key, sentence2_key = ("premise", "hypothesis") # for mnli

print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


Preprocessing

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        padding=True,
        truncation=True
    )
preprocess_function(dataset['train'][:5])

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e918085800bdaf35.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-19eaff48a7946f26.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fbc8360796e1193d.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d767475232a1637b.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-61b7eb49a8b6e726.arrow


In [ ]:
args = TrainingArguments(
    f"{MODEL_NAME}-finetuned-{optimizer_name}-mnli",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=6.7e-5,#6.7e-5(81.7),#1.8e-5,#2.4e-5 , #3e-5 AdamW 
    per_device_train_batch_size=BATCHSIZE,
    per_device_eval_batch_size=BATCHSIZE,
    num_train_epochs=EPOCHNUM,                                          
    weight_decay=WEIGHT_DECAY,                                               
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    push_to_hub=True,
    report_to="wandb",
)
#accuracy score 82.4    83.1   81.7   82.17    81.7
#lr             6.7e-5  6.7e-5 6.7e-5 6.7e-5   6.7e-5   6.7e-4     6.7e-4  6.7e-4
#weight_decay   1.14  1.2ou1.1 1.0    0.8      0.8      0.01       0.5

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# #estimated_training_size =args.size_of_dataset if args.size_of_dataset is not None else len(encoded_dataset["train"])
# estimated_training_size = len(encoded_dataset["train"])
# num_epoch = args.num_train_epochs*100 if args.num_train_epochs is not None else 1

# print(num_epoch, estimated_training_size,args.num_train_epochs)
# total_step = estimated_training_size * num_epoch // args.per_device_train_batch_size
# if total_step <= 0:
#         t_total = estimated_training_size * num_epoch // args.actual_train_batch_size
# else:
#     t_total = total_step

# if args.warmup_steps <= 0:  # set the warmup steps to 0.1 * total step if the given warmup step is -1.
#     args.warmup_steps = int(t_total * 0.1)
# no_decay = ["bias", "LayerNorm.weight"]
# optimizer_grouped_parameters = [
#     {
#         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#         "weight_decay": args.weight_decay,
#     },
#     {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
# ]

# optimizer = OPTIMIZER(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
# #optimizer = SGD(optimizer_grouped_parameters, lr=args.learning_rate, momentum=0.0, weight_decay=args.weight_decay) #does not get satisfying results lr=3e-6 3e-5 3e-4 3e-3 3e-2
# #optimizer=Adadelta(optimizer_grouped_parameters, lr=args.learning_rate, weight_decay=args.weight_decay,) #lr=3e-6 #does not get satisfying results either
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
# )

20.0 392702 0.2


In [ ]:
trainer = Trainer(
    MODEL,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation_matched"],#for mnli
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers=(optimizer, scheduler),
    #optimizers=(transformers.AdamW, transformers.get_scheduler("linear", num_warmup_steps=0, num_training_steps=args.num_train_epochs*len(encoded_dataset["train"])/args.per_device_train_batch_size)),
    #token=token
)

/home/nico/Documents/IAS2/IAssistant/AI-base/mpnet-base-finetuned-AdamW-mnli is already a clone of https://huggingface.co/NicolasLe/mpnet-base-finetuned-AdamW-mnli. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: hypothesis, premise, idx. If hypothesis, premise, idx are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
/home/nico/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 392702
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 19636
  Number of trainable parameters = 109488771
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] =

  0%|          | 0/19636 [00:00<?, ?it/s]

You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.0972, 'learning_rate': 1.7061283110348304e-07, 'epoch': 0.01}
{'loss': 1.0993, 'learning_rate': 3.412256622069661e-07, 'epoch': 0.01}
{'loss': 1.0991, 'learning_rate': 5.118384933104492e-07, 'epoch': 0.02}
{'loss': 1.0964, 'learning_rate': 6.824513244139321e-07, 'epoch': 0.02}
{'loss': 1.0964, 'learning_rate': 8.530641555174153e-07, 'epoch': 0.03}
{'loss': 1.0885, 'learning_rate': 1.0236769866208984e-06, 'epoch': 0.03}
{'loss': 1.0412, 'learning_rate': 1.1942898177243815e-06, 'epoch': 0.04}
{'loss': 0.9571, 'learning_rate': 1.3649026488278643e-06, 'epoch': 0.04}
{'loss': 0.8847, 'learning_rate': 1.5355154799313475e-06, 'epoch': 0.05}
{'loss': 0.82, 'learning_rate': 1.7061283110348306e-06, 'epoch': 0.05}
{'loss': 0.7683, 'learning_rate': 1.8767411421383134e-06, 'epoch': 0.06}
{'loss': 0.7126, 'learning_rate': 2.047353973241797e-06, 'epoch': 0.06}
{'loss': 0.7062, 'learning_rate': 2.2179668043452795e-06, 'epoch': 0.07}
{'loss': 0.664, 'learning_rate': 2.388579635448763e-06, 'e

In [ ]:
trainer.push_to_hub(model_name=f"{MODEL_NAME}-finetuned-{optimizer_name}-mnli")


Saving model checkpoint to mpnet-base-finetuned-AdamW-mnli
Configuration saved in mpnet-base-finetuned-AdamW-mnli/config.json
Model weights saved in mpnet-base-finetuned-AdamW-mnli/pytorch_model.bin
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/NicolasLe/mpnet-base-finetuned-AdamW-mnli
   a193af4..46ef91a  main -> main



: 

In [ ]:
# #fitetuningpretrained pytorch from scratch
# del model
# del trainer
# del args
# #del batch
# torch.cuda.empty_cache()